In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb
import shutil

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

In [2]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

In [3]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

(241, 41)
Index(['lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'n_val', 'train',
       'y_low', 'device', 'logger', 'n_test', 'run_id', 'y_high', 'log_dir',
       'n_train', 'predict', 'version', 'data_dir', 'eps_mult', 'n_splits',
       'criterion', 'do_return', 'earlystop', 'grid_size', 'noise_std',
       'only_flip', 'batch_size', 'max_epochs', 'data_kwargs', 'num_workers',
       'generate_data', 'model_summary', 'args_processed', 'do_return_model',
       'transform_kwargs', 'persistent_workers', 'run_name', 'test_loss',
       'space_length'],
      dtype='object')


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,space_length
0,0.0001,Predict-TrainedP,False,None,3,[sine2d-rot-predict],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,virtuous-fuse-614,0.001333,NaN
1,0.0001,Predict-NoneP,False,None,3,[sine2d-rot-predict],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,sparkling-paper-613,0.002179,NaN
2,0.0001,Predict-TrainedP,False,None,2,[sine2d-rot-predict],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,crimson-peony-611,0.001435,NaN
3,0.0001,Predict-NoneP,False,None,2,[sine2d-rot-predict],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,glowing-dumpling-612,0.002166,NaN
4,0.0010,Predict-TrainedP,False,None,3,[sine2d-rot-predict],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,red-springroll-610,0.001172,NaN


In [4]:
api = wandb.Api()
runs = api.runs('eliasdubbeldam/symlie')
run_id_api = {run.id : {'tags':run.tags, 'name':run.name} for run in runs}
print(len(run_id_api))
run_id_api

309


{'9imrjohj': {'tags': ['flower'], 'name': 'filigreed-peony-642'},
 '7ihsvxqq': {'tags': ['flower'], 'name': 'sweet-rocket-641'},
 'm4mogsm6': {'tags': ['flower'], 'name': 'incandescent-rabbit-640'},
 'tyrndo7h': {'tags': ['flower'], 'name': 'red-wonton-637'},
 'm3rulu9d': {'tags': ['flower'], 'name': 'beaming-kumquat-638'},
 '39gn7v1k': {'tags': ['flower'], 'name': 'dancing-envelope-639'},
 'iz6elw9k': {'tags': ['flower'], 'name': 'glistening-fish-636'},
 '0g7p8uf0': {'tags': ['flower'], 'name': 'lucky-rocket-635'},
 'grip6095': {'tags': ['flower'], 'name': 'beaming-dragon-631'},
 '6dxxdwc2': {'tags': ['flower'], 'name': 'dazzling-moon-633'},
 'w480if8b': {'tags': ['flower'], 'name': 'virtuous-laughter-633'},
 'f9fuzrg7': {'tags': ['flower'], 'name': 'vibrant-cake-631'},
 'oelcb39k': {'tags': ['flower'], 'name': 'red-snake-630'},
 '2zd52kzq': {'tags': ['flower'], 'name': 'alight-laughter-629'},
 '9o8pa5tr': {'tags': ['flower'], 'name': 'lucky-lamp-627'},
 '7mkvujza': {'tags': ['flower'

In [14]:
tidy_dirs = {
    'P': {
        'dir'     : os.path.join(store_dir, 'P'), 
        'listdir' : lambda dir: [(dir, dir.split('.')[0]) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: os.remove(dir),
    },
    'symlie' : {
        'dir'     : os.path.join(log_dir, 'symlie'), 
        'listdir' : lambda dir: [(dir, dir) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
    'wandb' : {
        'dir'     : os.path.join(wandb_dir), 
        'listdir' : lambda dir: [(dir, dir.split('-')[-1]) for dir in os.listdir(dir) if dir.startswith('run')],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
}

REMOVE_RUNS = True
VERBOSE     = False

for tidy_dir, t in tidy_dirs.items():
    count = 0
    for dir_full, dir_strip in t['listdir'](t['dir']):
        if dir_strip not in run_id_api.keys():
            if VERBOSE: 
                print(f'Deleting {dir_full}')
            count += 1
            if REMOVE_RUNS:
                t['rmdir'](os.path.join(t['dir'], dir_full))
    print(f'{count} directories removed from {tidy_dir}')

0 directories removed from P
43 directories removed from symlie
43 directories removed from wandb
